In [ ]:
# 1. Do this the first time only to install dependencies
!pip install tensorflow opencv-python scikit-learn numpy mediapipe

In [1]:
# 2
import cv2
import tensorflow as tf
import mediapipe as mp
import os
import numpy as np
from matplotlib import pyplot as plt
import time

## Initializing Detection Model and Drawing Util

In [3]:
# 3
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

## Defining detection model and drawing tool

In [4]:
# 4
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image, results

In [5]:
# 5
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

## Execution Test

In [ ]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while True:
        ret, frame = cap.read()
    
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image, results)
        
        cv2.imshow("OpenCV Feed", image)
        
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()

## Extracting keypoints

In [6]:
# 6
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([lh, rh])

## Directories for collecting Data

In [7]:
# 7
data_dir = os.path.join('data')

# J and Z are not includede (motion based alphabets)
letters = np.array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'])  
no_sequences = 25                   # (25 lh, 25rh) = total(50)
sequence_length = 10

In [ ]:
for letter in letters:
    # for sequence in range(25, 50):   [use this for the first half of the data(eg: left hand[25] then right hand[25])]
    for sequence in range(0, 25):
        try:
            os.makedirs(os.path.join(data_dir, letter, str(sequence)))
        except:
            pass

## Collecting data from the webcam

In [ ]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for letter in letters:
        flag = True
        # for sequence in range(0, 24):      [use this for the first hand keypoints]
        for sequence in range(0, 25):
            for frame_number in range(sequence_length):
                ret, frame = cap.read()
            
                image, results = mediapipe_detection(frame, holistic)
                draw_landmarks(image, results)
                
                if frame_number == 0:
                    while(flag):
                        cv2.putText(image, 'START Collection for letter \'{}\', press S.'.format(letter), (120, 200), 
                                cv2.FONT_HERSHEY_COMPLEX, 1,(0, 255, 0), 1, cv2.LINE_AA)
                        cv2.imshow("OpenCV Feed", image)
                        if cv2.waitKey(1) & 0xFF == ord('s'):
                            flag = False
                    cv2.putText(image, 'Collecting for letter \'{}\' video number {}'.format(letter, sequence), (120, 200), 
                                cv2.FONT_HERSHEY_COMPLEX, 1,(0, 255, 0), 1, cv2.LINE_AA)
                else:
                    cv2.putText(image, 'Collecting for letter \'{}\' video number {}'.format(letter, sequence), (120, 200), 
                                cv2.FONT_HERSHEY_COMPLEX, 1,(0, 255, 0), 1, cv2.LINE_AA)

                keypoints = extract_keypoints(results)
                np_path = os.path.join(data_dir, letter, str(sequence), str(frame_number))
                np.save(np_path, keypoints)
                
                cv2.imshow("OpenCV Feed", image)
                
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

## Preprocess the collected data

In [8]:
## 1
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
## 2
label_map = {label: num for num, label in enumerate(letters)}

In [10]:
## 3
sequences, labels = [], []
for letter in letters:
    for sequence in range(no_sequences):
        window = []
        for frame_number in range(sequence_length):
            res = np.load(os.path.join(data_dir, letter, str(sequence), '{}.npy'.format(frame_number)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[letter])

In [11]:
np.array(sequences).shape

(600, 10, 126)

In [12]:
np.array(labels).shape

(600,)

In [ ]:
X = np.array(sequences)
X.shape

In [ ]:
y = to_categorical(labels).astype(int)
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
y_test.shape

## Build and Train LSTM model

In [13]:
# 8
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [14]:
# 9
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [15]:
# 10
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(10, 126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(letters.shape[0], activation='softmax'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 10, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 24)                792       
                                                                 
Total params: 204152 (797.47 KB)
Trainable params: 20415

In [17]:
# 11
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [18]:
# 12
model.load_weights("asl.h5")

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

## Make predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
letters[np.argmax(res[29])]

In [ ]:
letters[np.argmax(y_test[29])]

In [ ]:
np.argmax(res[0])

## Save Model

In [ ]:
model.save('asl.h5')
model.save('asl.keras')

## Evaluate the Model

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

## Test Model

In [ ]:
# 13 Main execution part

predictions = []
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
    
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-10:]

        if len(sequence) == 10:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(letters[np.argmax(res)])
            predictions.append(np.argmax(res))

            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    letter = letters[np.argmax(res)]
                else:
                    letter = 'no input'

        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, letter, (3, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow("ASL Recognition", image)
        
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 1s 737ms/step
p
1/1 [==============================] - 0s 16ms/step
p
1/1 [==============================] - 0s 17ms/step
p
1/1 [==============================] - 0s 33ms/step
p
1/1 [==============================] - 0s 20ms/step
p
1/1 [==============================] - 0s 32ms/step
p
1/1 [==============================] - 0s 32ms/step
p
1/1 [==============================] - 0s 8ms/step
p
1/1 [==============================] - 0s 16ms/step
p
1/1 [==============================] - 0s 16ms/step
p
1/1 [==============================] - 0s 32ms/step
p
1/1 [==============================] - 0s 16ms/step
p
1/1 [==============================] - 0s 15ms/step
p
1/1 [==============================] - 0s 16ms/step
p
1/1 [==============================] - 0s 17ms/step
p
1/1 [==============================] - 0s 16ms/step
p
1/1 [==============================] - 0s 16ms/step
p
1/1 [==============================] - 0s 16ms/step
p
1/1 [=======================

In [ ]:
## 14 --- Use this in case webcam is still running after closing the program 
cap.release()
cv2.destroyAllWindows()